In [1]:
import numpy as np  
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

In [2]:
class Layer:
    def forward(self,X):
        pass
    
    def backward(self,E):
        pass

class Dense(Layer):
    def __init__(self,input_size, output_size):
        self.w = np.random.rand(input_size+1, output_size)
    def forward(self,X):
        # add bias
        X = np.hstack((X,np.ones((len(X),1))))
        self.X = X
        return X @ self.w
    
    def backward(self,E):
        prev_error = E @ self.w[:-1].T
        self.w -= self.X.T @ E
        return prev_error

class Conv2D(Layer):
    def __init__(self,height,width,channel,filter_num,filter_height,filter_width,stride,padding=(0,0)):
        '''
        height: height of the image
        width:  width of the image
        filter_num: number of filters to use. The size of the output channel should be
                    the same as filter_num
        filter_size: size of the filter, each filter has shape filter_size x filter_size
        stride: pixels to move between each convolution
        padding: zero paddings for height and width padding has shape: (height_padding, width_padding)
        '''
        # copy parameters
        self.height = height
        self.width = width
        self.channel = channel
        self.filter_num = filter_num
        self.filter_height = filter_height
        self.filter_width = filter_width
        self.stride = stride
        self.padding = padding
        # end of copying parameters
        
        self.paddingh = padding[0]
        self.paddingw  = padding[1]
        # initialize filters and corresponding biases
        self.filters = np.random.rand(filter_num, channel, filter_height, filter_width)
        self.biases  = np.random.rand(filter_num, 1)
        # compute output shape for one layer of an image
        self.output_height = (height+self.paddingh - filter_height) // stride + 1
        self.output_width =  (width + self.paddingw - filter_width) // stride + 1
        
    def forward(self,X):
        '''
        X: input of images.
        We assume X is a numpy array with shape (N, C, H, W)
                 N        number of images
                 H   the height of one image
                 W    the width of one image
                 C  depth of one image. Like how many layers stacked together
        '''
        N,C,H,W = X.shape
        assert H == self.height and W== self.width
        # add padding
        X = np.pad(X,((0,0),(0,0),(0,self.padding[0]),(0,self.padding[1])),'constant',constant_values=(0, 0))
        # preallocate the output
        output = np.zeros(X.shape[0],self.filter_num,self.output_height,self.output_width)
        
        for n in range(N): # process each image
            for f in range(self.filter_num): # iterate through each filter
                image = X[n][c]
                for c in range(C): # process each channel
                    for i in range(self.output_height): # each stride down
                        for j in range(self.output_width): # each stride left
                            height_start = i * self.stride
                            width_start = j * self.stride
                            region = image[height_start:height_start+self.filter_height,width_start:width_start + self.filter_width]
                            output[n][f][i][j] += np.sum(region * self.filters[f])
        return output
    
    def backward(self,E):
        pass
class MaxPooling(Layer):
    def forward(self,X):
        pass
    
    def backward(self,E):
        pass
    
class Relu(Layer):
    def forward(self,X):
        self.derivative = X > 0
        # fancy index  
#         X[X<0] = 0
        return X * (X>0)
    def backward(self,E):
        return E * self.derivative
    
class SoftMax(Layer):
    def forward(self,X):
        X_exp = np.exp(X)
        X_exp_sum = np.sum(X_exp,axis=1).reshape(-1,1)
        return X_exp / X_exp_sum
    # assume we have the matching loss
    # then the error for last layer would
    # just be y - t
    def backward(self,E):
        return E
    
def toPredict(predict):
    result = []
    for row in predict:
        max_v = 0
        r_row = []
        for x in row:
            max_v = max(max_v,x)
        for x in row:
            r_row.append(1) if x == max_v else r_row.append(0)
        result.append(r_row)
    return np.array(result)
                

In [3]:
x = np.random.random((2,3))
print(x)
print(toPredict(x))

[[0.72881332 0.50950789 0.78707078]
 [0.00404911 0.241886   0.86768526]]
[[0 0 1]
 [0 0 1]]


In [5]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
iris = datasets.load_iris()
X = iris.data# we only take the first two features.
y = iris.target
Y = np.zeros((y.size, 3))
Y[np.arange(y.size),y] = 1

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



layers = [Dense(X.shape[1], 10), Relu(), Dense(10,3),SoftMax()]

max_ite = 100
alpha = 0.0001

for i in range(1000):
    train_X = X_train
    for layer in layers:
        train_X = layer.forward(train_X)
        #print(train_X)
    E = (train_X - y_train) * alpha
    for k in range(len(layers)-1,-1,-1):
        E = layers[k].backward(E)
    #print(E)
for layer in layers:
    X_test = layer.forward(X_test)
y_pred = toPredict(X_test)
accuracy_score(y_test, y_pred)


0.9666666666666667